In [27]:
import numpy as np
import numpy.random as random
import pandas as pd
from pandas import Series, DataFrame
from datetime import datetime, timedelta
import os
import re

In [ ]:
# 基本的に以下を行えば準備完了

# 前処理を行うファイルを./data/xxxxxxxxtestx/InsoleSensor/0_defaultに用意する
# 前処理を行うファイルを./data/xxxxxxxxtestx/Opti-track/0_defaultに用意する
# ここでパスを指定する
BASE_DIR = './data/20250518test4'

## P2P-Insole前処理用

In [ ]:
# データクリーナー1
# -方向データの削除
# -データを0.01秒刻みに整形

data_path = BASE_DIR + '/InsoleSensor/0_default'
output_path = os.path.join(data_path, '../1_data_clean1')

# 出力先ディレクトリが存在しない場合は作成
os.makedirs(output_path, exist_ok=True)

csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

def resample_and_interpolate(data):
    # タイムスタンプ列の名前を明示的に指定
    timestamp_col = 'Timestamp'
    if timestamp_col not in data.columns:
        print(f"エラー: タイムスタンプ列 '{timestamp_col}' がデータに存在しません。")
        return None

    # タイムスタンプをdatetimeオブジェクトに変換し、インデックスに設定
    # 数値（秒）として保存されていると仮定し、基準日（エポック）からの経過時間として処理
    data[timestamp_col] = pd.to_datetime(data[timestamp_col], unit='s', origin='unix')
    data = data.set_index(timestamp_col)

    # 重複したインデックスを処理 (最初の出現を保持)
    data = data[~data.index.duplicated(keep='first')]

    # 0.01秒刻みでリサンプリングし、欠損値を線形補間
    data_resampled = data.resample('10ms').mean().interpolate(method='linear')

    return data_resampled

for csv_file in csv_files:
    # CSVファイルの読み込み(header=1 <=> 一行目の//DN:108,SN:35の行を削除)
    file_path = os.path.join(data_path, csv_file)
    data = pd.read_csv(file_path, header=1)

    # 方位データを削除
    if all(col in data.columns for col in ["Mag_x", "Mag_y", "Mag_z"]):
        data = data.drop(["Mag_x", "Mag_y", "Mag_z"], axis=1)

    # リサンプリング処理の適用
    data_resample = resample_and_interpolate(data.copy()) # 元のデータを変更しないようにコピーを渡す

    if data_resample is not None:
        # 出力ファイルの保存
        output_file_path = os.path.join(output_path, csv_file)
        data_resample.to_csv(output_file_path)

        # 処理内容の確認（例として最初の数行を表示）
        print(f'Processed {csv_file}:')
        print(data_resample.head())
    else:
        print(f'Error processing {csv_file}. スキップします。')

Processed 20241212_152700_left.csv:
                               P1        P2    P3   P4        P5        P6  \
Timestamp                                                                    
2024-12-12 06:07:01.920  1.398673  1.323773  50.0  0.0  0.000000  1.506013   
2024-12-12 06:07:01.930  1.526824  1.211876  50.0  0.0  0.014007  1.280941   
2024-12-12 06:07:01.940  1.538280  1.196996  50.0  0.0  0.000000  1.566848   
2024-12-12 06:07:01.950  1.521115  1.323773  50.0  0.0  0.013399  1.530120   
2024-12-12 06:07:01.960  1.509734  1.328975  50.0  0.0  0.014007  1.506013   

                               P7        P8        P9       P10  ...  \
Timestamp                                                        ...   
2024-12-12 06:07:01.920  1.304286  2.249370  0.281591  3.623655  ...   
2024-12-12 06:07:01.930  1.510561  2.249370  0.273090  4.399258  ...   
2024-12-12 06:07:01.940  1.282834  2.032406  0.273090  4.423294  ...   
2024-12-12 06:07:01.950  1.518672  2.249370  0.269751  4.

In [ ]:
# データクリーナー2
# -左右のタイムスタンプを一致させる

# フォルダのパスをここに指定してください
data_path = BASE_DIR + '/InsoleSensor/1_data_clean1' 
output_path = os.path.normpath(os.path.join(data_path, "../2_data_clean2"))
os.makedirs(output_path, exist_ok=True)

def sync_csv_by_timestamp(data_path, output_path):
    all_files = os.listdir(data_path)
    left_files = [f for f in all_files if '_left.csv' in f]

    for left_file in left_files:
        match = re.match(r'(.+)_left\.csv', left_file)
        if match:
            base_name = match.group(1)
            right_file = f"{base_name}_right.csv"

            if right_file in all_files:
                left_path = os.path.join(data_path, left_file)
                right_path = os.path.join(data_path, right_file)

                try:
                    left_df = pd.read_csv(left_path)
                    right_df = pd.read_csv(right_path)

                    if 'Timestamp' not in left_df.columns or 'Timestamp' not in right_df.columns:
                        print(f"'{left_file}' または '{right_file}' に 'Timestamp' カラムが存在しません。")
                        continue

                    left_df.set_index('Timestamp', inplace=True)
                    right_df.set_index('Timestamp', inplace=True)

                    common_timestamps = left_df.index.intersection(right_df.index)

                    if common_timestamps.empty:
                        print(f"'{left_file}' と '{right_file}' に共通のタイムスタンプがありません。")
                        continue

                    synced_left_df = left_df.loc[common_timestamps]
                    synced_right_df = right_df.loc[common_timestamps]

                    synced_left_path = os.path.join(output_path, f"{base_name}_left.csv")
                    synced_right_path = os.path.join(output_path, f"{base_name}_right.csv")

                    synced_left_df.to_csv(synced_left_path)
                    synced_right_df.to_csv(synced_right_path)

                    print(f"'{left_file}' と '{right_file}' を同期し、'{synced_left_path}' と '{synced_right_path}' に保存しました。")

                except Exception as e:
                    print(f"エラー: '{left_file}' と '{right_file}' の処理中にエラーが発生しました: {e}")
    
sync_csv_by_timestamp(data_path, output_path)
print("処理が完了しました。")

'20241212_152700_left.csv' と '20241212_152700_right.csv' を同期し、'data\20250518test4\InsoleSensor\2_data_clean2\20241212_152700_left.csv' と 'data\20250518test4\InsoleSensor\2_data_clean2\20241212_152700_right.csv' に保存しました。
'20241212_160501_left.csv' と '20241212_160501_right.csv' を同期し、'data\20250518test4\InsoleSensor\2_data_clean2\20241212_160501_left.csv' と 'data\20250518test4\InsoleSensor\2_data_clean2\20241212_160501_right.csv' に保存しました。
'20241212_164800_left.csv' と '20241212_164800_right.csv' を同期し、'data\20250518test4\InsoleSensor\2_data_clean2\20241212_164800_left.csv' と 'data\20250518test4\InsoleSensor\2_data_clean2\20241212_164800_right.csv' に保存しました。
'20241212_173800_left.csv' と '20241212_173800_right.csv' を同期し、'data\20250518test4\InsoleSensor\2_data_clean2\20241212_173800_left.csv' と 'data\20250518test4\InsoleSensor\2_data_clean2\20241212_173800_right.csv' に保存しました。
処理が完了しました。


In [ ]:
# P2P-insoleのタイムスタンプがUTC(協定世界時)に合わせられている場合はこのコードを実行する

# フォルダのパスをここに指定してください
data_path = BASE_DIR + '/InsoleSensor/2_data_clean2' 
output_path = BASE_DIR + '/InsoleSensor/2_data_clean2' 

csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

def convert_utc_to_jst(csv_file_path):
    # CSVファイルを読み込む
    df = pd.read_csv(csv_file_path)

    # タイムスタンプ列をdatetime型に変換 (UTCとして認識)
    df['Timestamp'] = pd.to_datetime(df['Timestamp'], utc=True)

    # UTCからJSTに変換 (+9時間)
    df['Timestamp'] = df['Timestamp'].dt.tz_convert('Asia/Tokyo')

    return df
    

for csv_file in csv_files:
    # CSVファイルの読み込み
    file_path = os.path.join(data_path, csv_file)
    data = pd.read_csv(file_path)

    # UTCからJSTに変換
    converted_df = convert_utc_to_jst(file_path)
   
    # 出力ファイルの保存
    output_file_path = os.path.join(output_path, csv_file)
    converted_df.to_csv(output_file_path, index=False)   


## Opti-Track前処理用

In [ ]:
# データクリーナー1
# -Frame, 偶数ポイント行の削除
# -データ頭の詳細情報を削除
# -ボーン以外のデータを削除するコード(X_42以降)

# 0_defaultフォルダに生データを入れる
data_path = BASE_DIR + '/Opti-track/0_default'
output_path = os.path.normpath(os.path.join(data_path, "../1_data_clean1"))
os.makedirs(output_path, exist_ok=True)

# フォルダ内のすべてのCSVファイルを取得
csv_files = [f for f in os.listdir(data_path) if f.endswith('.csv')]

# CSVファイルごとに処理を行う
for csv_file in csv_files:
    # CSVファイルを読み込み
    file_path = os.path.join(data_path, csv_file)
    data = pd.read_csv(file_path, header = 5)

    data = data.drop(['Frame','X','Y','Z'],axis=1)
    for n in range(2, 41, 2):
        data = data.drop(['X.'+ str(n), 'Y.'+ str(n), 'Z.'+ str(n)], axis=1)

    for n in range(42, 500, 1):
        try:
            data = data.drop(['X.'+ str(n), 'Y.'+ str(n), 'Z.'+ str(n)], axis=1)
        except KeyError :
            print(['X.'+ str(n), 'Y.'+ str(n), 'Z.'+ str(n)], "not found in axis")
            break
    
    data.to_csv(output_path + "/" +csv_file, index=False)

,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.306,X.307,Y.307,Z.307,X.308,Y.308,Z.308,X.309,Y.309,Z.309
0,0,0.00,-1.620359,-2.373934,-1.223901,51.242870,858.580261,-451.291809,-9.843589,-2.093895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.01,-1.409878,-1.292866,-0.558986,53.595749,863.872559,-434.481201,-9.705350,-0.889141,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.02,-1.757537,-0.397078,-0.557272,61.415115,864.064514,-419.504456,-12.546810,-0.193872,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.03,-2.048083,0.270302,-0.628513,64.830666,861.183105,-412.194550,-14.987641,0.094877,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.04,-2.120459,0.834467,-0.603021,63.929878,858.542358,-410.176971,-16.396343,0.274675,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['X.310', 'Y.310', 'Z.310'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.271,X.272,Y.272,Z.272,X.273,Y.273,Z.273,X.274,Y.274,Z.274
0,0,0.00,-0.591041,-1.046665,1.419387,2.895983,879.538818,-218.747253,-8.457394,-1.157118,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.01,-0.097568,0.755583,0.609421,11.492421,873.408997,-194.576019,-7.396871,0.703555,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.02,-0.392287,2.753636,-0.250060,17.411655,868.279541,-177.059677,-9.675610,2.638838,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.03,-0.896700,4.190225,-0.936388,16.847097,863.030457,-169.496979,-11.866702,3.853897,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.04,-1.329052,5.113201,-1.306284,13.014276,859.188721,-167.583633,-13.110983,4.386281,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


['X.275', 'Y.275', 'Z.275'] not found in axis


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.476,X.477,Y.477,Z.477,X.478,Y.478,Z.478,X.479,Y.479,Z.479
0,0,0.00,-1.792345,-1.798784,0.552043,-12.973899,887.408997,-45.187340,-8.676855,-2.010712,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.01,-2.307702,-4.598279,0.369096,-6.544707,887.501221,-45.174446,-9.890988,-5.402292,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.02,-2.963188,-6.339859,0.024253,-0.246855,885.454895,-41.642616,-11.938392,-7.060720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.03,-3.250772,-7.214124,-0.193652,2.373243,881.691040,-38.457756,-13.378972,-7.461792,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.04,-3.180672,-7.719501,-0.074362,2.059535,877.819153,-37.595852,-14.111599,-7.604745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Frame,Time (Seconds),X,Y,Z,X.1,Y.1,Z.1,X.2,Y.2,...,Z.467,X.468,Y.468,Z.468,X.469,Y.469,Z.469,X.470,Y.470,Z.470
0,0,0.00,-0.033485,-9.265938,1.057669,-22.597157,927.108276,313.754913,-9.456438,-8.754055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,0.01,0.615172,-5.546481,0.953289,-22.154085,924.821045,344.691345,-7.935656,-3.648250,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0.02,-0.011496,-3.043415,0.525539,-19.740488,922.192993,367.434021,-11.783314,-0.416126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,0.03,-1.045944,-1.632664,0.150387,-21.116028,918.039001,375.169098,-15.409142,1.187843,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,0.04,-1.920458,-0.702325,-0.095779,-23.839668,914.773804,376.024567,-17.427410,2.084630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# データクリーナー2
# タイムスタンプを作成するコード
data_path = BASE_DIR + '/Opti-track/1_data_clean1'
output_path = os.path.normpath(os.path.join(data_path, "../2_data_clean2"))
os.makedirs(output_path, exist_ok=True)

def convert_time_seconds_to_timestamp(filename):
    try:
        # CSVファイルを読み込む
        df = pd.read_csv(filename)
    except FileNotFoundError:
        print(f"エラー: ファイル '{filename}' が見つかりません。")
        return None
    except Exception as e:
        print(f"エラー: CSVファイルの読み込み中にエラーが発生しました: {e}")
        return None

    try:
        # 正規表現でファイル名から日時とAM/PMを抽出
        base_filename = os.path.basename(filename)
        match = re.search(r'(\d{4}-\d{2}-\d{2})\s+(\d{2})\.(\d{2})\.(\d{2})\s+(AM|PM)', base_filename)
        if match:
            date_part = match.group(1).replace('-', '/')
            hour_part = int(match.group(2))
            minute_part = int(match.group(3))
            second_part = int(match.group(4))
            am_pm_part = match.group(5)
            start_datetime = datetime(int(date_part.split('/')[0]),
                                      int(date_part.split('/')[1]),
                                      int(date_part.split('/')[2]),
                                      hour_part, minute_part, second_part)
            if am_pm_part == 'PM' and hour_part != 12:
                start_datetime = start_datetime + timedelta(hours=12)
            elif am_pm_part == 'AM' and hour_part == 12:
                start_datetime = start_datetime - timedelta(hours=12) # midnight case

        else:
            print(f"エラー: ファイル '{base_filename}' から日時を抽出できませんでした。ファイル名が予期された形式ではありません。")
            return df

    except ValueError as e:
        print(f"エラー: ファイル '{base_filename}' から抽出した日時を解析できませんでした: {e}")
        return df  # ファイルは読み込めたので、変換前のDataFrameを返す

    # Time (Second) 列をtimedeltaオブジェクトに変換
    try:
        time_deltas = pd.to_timedelta(df['Time (Seconds)'], unit='s')
    except KeyError:
        print(f"エラー: ファイル '{base_filename}' に 'Time (Seconds)' という列が見つかりません。")
        return df  # 'Time (Second)' 列がない場合は、元のDataFrameを返す

    # 開始日時にtimedeltaを加算してTimestamp列を生成
    timestamp_column = start_datetime + time_deltas

    # データフレームにTimestamp列を追加
    df['Time (Seconds)'] = timestamp_column
    new_df = df.rename(columns={'Time (Seconds)' : 'Timestamp'})

    # # Time (Second) 列を削除
    # if 'Time (Second)' in df.columns:
    #     df = df.drop(columns=['Time (Seconds)'])
    # else:
    #     print(f"警告: ファイル '{base_filename}' に 'Time (Seconds)' 列が見つかりませんでした。")

    # # Timestamp 列を先頭に移動
    # timestamp_series = df.pop('Timestamp')
    # df.insert(0, 'Timestamp', timestamp_series)

    return new_df

def process_all_csv_in_folder(folder_path):
    if not os.path.isdir(folder_path):
        print(f"エラー: 指定されたパス '{folder_path}' はフォルダではありません。")
        return

    for filename in os.listdir(folder_path):
        if filename.endswith(".csv"):
            full_filepath = os.path.join(folder_path, filename)
            print(f"処理中のファイル: {full_filepath}")
            df_with_timestamp = convert_time_seconds_to_timestamp(full_filepath)
            if df_with_timestamp is not None:
                try:
                    df_with_timestamp.to_csv(os.path.join(output_path, filename), index=False)
                    print(f"処理後のファイルは '{output_path}' に保存されました。")
                except Exception as e:
                    print(f"エラー: 処理後のファイルの保存中にエラーが発生しました: {e}")
            print("-" * 30)


# フォルダ内のすべてのCSVファイルを処理
process_all_csv_in_folder(data_path)
print("処理が完了しました。")

処理中のファイル: ./data/20250518test4/Opti-track/1_data_clean1\Take 2024-12-12 03.06.59 PM.csv
処理後のファイルは 'data\20250518test4\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250518test4/Opti-track/1_data_clean1\Take 2024-12-12 03.45.00 PM.csv
処理後のファイルは 'data\20250518test4\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250518test4/Opti-track/1_data_clean1\Take 2024-12-12 04.28.00 PM.csv
処理後のファイルは 'data\20250518test4\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理中のファイル: ./data/20250518test4/Opti-track/1_data_clean1\Take 2024-12-12 05.17.59 PM.csv
処理後のファイルは 'data\20250518test4\Opti-track\2_data_clean2' に保存されました。
------------------------------
処理が完了しました。


## P2P-insole, Optitrack共通処理

In [ ]:
# タイムスタンプ同期
# タイムスタンプ削除

In [ ]:
# nan値を変換
# スケーリング
# ガウスフィルター